In [23]:
import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Importa la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor
import os
import numpy as np
import networkx as nx
from sklearn.cluster import AffinityPropagation
from collections import Counter

# Aggiungi il percorso del modulo al sys.path


# Funzione per eseguire il clustering con Affinity Propagation
def affinity_propagation_clustering(graph):
    """
    Esegue il clustering sul grafo usando Affinity Propagation.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
    
    Returns:
        dict: Un dizionario con l'etichetta del cluster per ciascun nodo.
    """
    # Estrai la matrice di adiacenza come feature
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Utilizziamo la matrice di similarità (può essere la matrice di adiacenza stessa)
    similarity_matrix = adjacency_matrix
    
    # Inizializza e adatta il modello di Affinity Propagation
    ap = AffinityPropagation(affinity='precomputed', random_state=42)
    ap.fit(similarity_matrix)
    
    # Etichette dei cluster
    cluster_labels = ap.labels_
    
    # Restituiamo le etichette dei cluster per ciascun nodo
    return {node: label for node, label in zip(graph.nodes, cluster_labels)}

# Funzione per sanitizzare gli attributi del grafo
def sanitize_graph_attributes(graph):
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)

    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Esegui il clustering con Affinity Propagation
ap_labels = affinity_propagation_clustering(graph)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in ap_labels.items():
    gr.nodes[node]['cluster'] = label

# Sanitizza gli attributi del grafo prima di esportarlo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_affinity_propagation_clusters.gml"
nx.write_gml(gr, output_path)

# Visualizza il numero di nodi per ciascun cluster
cluster_counts = Counter(ap_labels.values())
print(f"Numero di nodi per cluster: {cluster_counts}")

print(f"Clustering con Affinity Propagation completato! Grafo esportato in: {output_path}")


ModuleNotFoundError: No module named 'sklearn'